In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import os
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet

In [3]:
def crop_image(image_path, threshold=0.95):
    image = Image.open(image_path)
    image = np.asarray(image)

    # detect face
    detector = MTCNN()
    results = [detection for detection in detector.detect_faces(image) if detection['confidence'] > threshold]

    message = ''
    face = None
    if len(results) == 0:
        message = 'No face detected'
    if len(results) > 1:
        message = 'More than one face is detected'
    if len(results) == 1:
        # crop image
        x1, y1, w, h = results[0]['box']
        x2, y2 = x1 + w, y1 + h
        
        face = image[y1:y2, x1:x2]

        # resize image
        face = Image.fromarray(face)
        face = face.resize((160,160))
        face = np.asarray(face)
        # transform face into one sample
        face = np.expand_dims(face, axis=0)

    return face, message

In [4]:
def load_csv(dataset):
    if os.path.exists(f'{dataset}.csv') == False: 
        # create empty table with 512 columns + name column
        columns = [i for i in range(512)]
        columns.append('name')
        df = pd.DataFrame(columns=columns)

        df.to_csv(f'{dataset}.csv',index=False)
    else:
        df = pd.read_csv(f'{dataset}.csv')
    
    return df

In [5]:
def register(image, name:str, dataset='registered_faces'):
    face, message = crop_image(image)

    if face is not None:
        embedding = FaceNet().embeddings(face)
        label = name.lower()
        df = load_csv(dataset)
        
        # add new embedding
        new_embedding = pd.DataFrame([embedding[0]], columns=df.columns[:-1])
        new_df= pd.concat([df.iloc[:,:-1], new_embedding], ignore_index=True)

        # add new name 
        label_list = []
        for i in df['name']:
            label_list.append(i)
        label_list.append(label)
        new_df['name']=label_list

        # save to csv
        new_df.to_csv(f'{dataset}.csv',index=False)

        message = 'face registered successfully'
    
    return message


In [6]:
def recognition(image, dataset='registered_faces', threshold=0.3):
    face, message = crop_image(image)

    if face is not None:
        embedding = FaceNet().embeddings(face)
        
        df = load_csv(dataset)
        x = df.iloc[:,:-1].values
        y = df['name']
        x_list = []
        y_list = []
        for i,j in zip(x,y):
            distance = FaceNet().compute_distance(i,embedding[0])
            x_list.append(distance)
            y_list.append(j)

        df_distance = pd.DataFrame({'distance':x_list, 'name':y_list})
        distance = df_distance.sort_values('distance')['distance'].iloc[0]

        if distance < threshold:
            message = df_distance.sort_values('distance')['name'].iloc[0]
        else:
            message = 'your face is not registered'
    
    return message

In [31]:
register('dataset/aa68282e-8d8a-4673-ad06-653db6f25dbf.jpg', 'Julio')

1/1 [==============================] - 3s 3s/step


'face registered successfully'

In [34]:
load_csv('registered_faces')

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,name
0,0.042070,-0.013130,-0.042852,0.061235,-0.013718,0.019664,-0.060343,-0.000904,-0.012497,-0.010789,...,-0.032241,0.001106,0.010889,0.022761,0.004176,-0.006886,0.005304,0.027033,0.022648,jackie
1,0.008973,0.037948,-0.020115,0.041850,0.047991,0.046354,0.069033,-0.009605,0.019609,-0.002764,...,-0.010195,0.025912,-0.052148,0.030658,0.046546,0.025084,-0.000999,-0.080459,0.040312,henry
2,0.048932,0.029114,-0.011183,0.081205,0.069276,0.017134,-0.014730,-0.021444,-0.030004,-0.019876,...,-0.000691,0.006172,-0.006311,0.019478,0.011630,-0.041529,-0.000824,0.067535,0.019101,julio
3,0.008973,0.037948,-0.020115,0.041850,0.047991,0.046354,0.069033,-0.009605,0.019609,-0.002764,...,-0.010195,0.025912,-0.052148,0.030658,0.046546,0.025084,-0.000999,-0.080459,0.040312,henry


In [33]:
recognition('dataset/aa68282e-8d8a-4673-ad06-653db6f25dbf.jpg')

1/1 [==============================] - 3s 3s/step


'julio'

## API Testing

In [7]:
import requests
import base64
url = 'http://localhost:5000'

In [97]:
# Register
# Open the image file
image_path = 'dataset/train/Alexander/Alexander_1.jpg'
name = 'Alexander'

with open(image_path, 'rb') as image_file:
    # Read the image data
    image_data = image_file.read()

# Convert the image data to base64-encoded string
image_base64 = base64.b64encode(image_data).decode('utf-8')

# Set the JSON payload
payload = {
    'image': image_base64,
    'name': name
    }

# Set the headers with 'Content-Type' as 'application/json'
headers = {'Content-Type': 'application/json'}

payload

{'image': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAsJCQcJCQcJCQkJCwkJCQkJCQsJCwsMCwsLDA0QDBEODQ4MEhkSJRodJR0ZHxwpKRYlNzU2GioyPi0pMBk7IRP/2wBDAQcICAsJCxULCxUsHRkdLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCz/wAARCADqASADASIAAhEBAxEB/8QAGwAAAgMBAQEAAAAAAAAAAAAAAAECBAYFAwf/xAA7EAABBAECBAQEBAQFBAMAAAABAAIDEQQSIQUxQVEGE2FxIjKBoRRCkbEjwdHhJDNy8PFDUpKyYmNz/8QAGQEBAAMBAQAAAAAAAAAAAAAAAAECAwQF/8QAJBEBAQACAwACAgEFAAAAAAAAAAECEQMhMRJBBGEiEyMyQlH/2gAMAwEAAhEDEQA/AMyOamEgN16AKEgdFMe6QCkAgYT+qAE0QApC+QRsuPl8YiYMqOGy8AxsfRADiKJ77IFxfijoNWLiuHm1U0go+Xf5WHlfc/7GZ1ONuJPqU3WT3PXupR26xpJ72iUKHXY2fUFGh9aq2adyrEcVnS4ECxRPbsrTMV4sblrmkH26InTmVeoepr6pURQ7q+3ClLjGQbB0/Xom7BmLS7Sb9LsFRs1XPBpSB3I9iFZOFOS34TVb7cu69G8PyPgOg7/L1J+ibhqqrJJGO1NcQeRrmFoOHcXg+TIdJ5hprdLQWUBy53ZXIfgTssaSq5ZJEbo+h5EKZZSyxuYciOcEtJG/Ijdeu/dZPh/FZoXRRytbIwua3VVSNBNc+y1jSHCxuFKoSs91JJEFv3RZ7ppIkWe6W/dNCIG/dFnuhNArPdFlNCAs90We6EIFZ7os90IUhWUb90IQVKUgEAKYH6KqwU6SAUgECAUqTAUJpoceN0krg1o79T2A7oKnFJTBhTPDtLiWsbXMlyx5dZ3JvnYIK6fE+

In [99]:
res = requests.post(url = f'{url}/register',json=payload, headers=headers)
print(res.json()) 

face registered successfully


In [11]:
pd.read_csv('registered_faces.csv')

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,name
0,0.042070,-0.013130,-0.042852,0.061235,-0.013718,0.019664,-0.060343,-0.000904,-0.012497,-0.010789,...,-0.032241,0.001106,0.010889,0.022761,0.004176,-0.006886,0.005304,0.027033,0.022648,jackie
1,0.008973,0.037948,-0.020115,0.041850,0.047991,0.046354,0.069033,-0.009605,0.019609,-0.002764,...,-0.010195,0.025912,-0.052148,0.030658,0.046546,0.025084,-0.000999,-0.080459,0.040312,henry
2,0.048932,0.029114,-0.011183,0.081205,0.069276,0.017134,-0.014730,-0.021444,-0.030004,-0.019876,...,-0.000691,0.006172,-0.006311,0.019478,0.011630,-0.041529,-0.000824,0.067535,0.019101,julio
3,0.008973,0.037948,-0.020115,0.041850,0.047991,0.046354,0.069033,-0.009605,0.019609,-0.002764,...,-0.010195,0.025912,-0.052148,0.030658,0.046546,0.025084,-0.000999,-0.080459,0.040312,henry
4,0.002933,0.054192,0.016526,0.052260,0.011469,-0.035688,0.013339,0.089709,0.030907,0.021558,...,-0.015220,-0.080756,0.037036,-0.004489,-0.047785,0.067489,-0.080729,-0.065167,0.008998,alexander
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,0.062223,-0.006538,-0.028069,0.047056,0.072438,0.098938,-0.002978,0.030283,-0.016256,0.007893,...,-0.058646,-0.052639,0.008654,0.032254,-0.000309,0.015705,-0.036552,-0.058592,-0.017187,simu
130,0.002933,0.054192,0.016526,0.052260,0.011469,-0.035688,0.013339,0.089709,0.030907,0.021558,...,-0.015220,-0.080756,0.037036,-0.004489,-0.047785,0.067489,-0.080729,-0.065167,0.008998,alexander
131,0.048240,0.028330,-0.010490,0.079738,0.069396,0.016188,-0.014091,-0.022619,-0.029410,-0.021139,...,-0.000053,0.007246,-0.005722,0.018866,0.010996,-0.041454,-0.001387,0.067531,0.017269,julio
132,0.048240,0.028330,-0.010490,0.079738,0.069396,0.016188,-0.014091,-0.022619,-0.029410,-0.021139,...,-0.000053,0.007246,-0.005722,0.018866,0.010996,-0.041454,-0.001387,0.067531,0.017269,julio


In [10]:
# Recognition
# Open the image file
image_path = 'Jonas Photo 3.jpg'

with open(image_path, 'rb') as image_file:
    # Read the image data
    image_data = image_file.read()

# Convert the image data to base64-encoded string
image_base64 = base64.b64encode(image_data).decode('utf-8')

# Set the JSON payload
payload = {'image': image_base64}

# Set the headers with 'Content-Type' as 'application/json'
headers = {'Content-Type': 'application/json'}

payload

{'image': '...'}

In [12]:
res = requests.post(url = f'{url}/recognition',json=payload, headers=headers)
print(res.json())

julio


## Program testing

In [76]:
train_path = 'dataset/train/'
names = os.listdir(train_path)

In [59]:
files =[]
for name in names:
    for file in os.listdir(train_path+name):
        register(train_path + name + '/' + file, name)

1/1 [==============================] - 2s 2s/step


In [62]:
predict = []
test_path = 'dataset/test/'
for file in os.listdir(test_path):
    predict.append(recognition(test_path + file))
 
predict

1/1 [==============================] - 2s 2s/step


['alexander',
 'andrew',
 'daniel',
 'harry',
 'henry',
 'jackie',
 'joe',
 'john',
 'justin',
 'kim',
 'lee',
 'lewis',
 'ludi',
 'manny',
 'max',
 'pang',
 'rafi',
 'rahul',
 'randall',
 'remy',
 'ronny',
 'ross',
 'simu',
 'steven',
 'vincent']

In [91]:
from sklearn.metrics import accuracy_score, f1_score

names_lower = [item.lower() for item in names]
accuracy = accuracy_score(np.array(names_lower), np.array(predict))
f1 = f1_score(names_lower, predict, average='macro')
print(f'Accuracy: {int(accuracy * 100)}%, F1 Score: {int(f1 * 100)}%')

Accuracy: 100%, F1 Score: 100%
